# Configure GeoLib to use Girder & NERSC resources

In [1]:
# Set NEWT session id
newt_sessionid = '57311ce617ff392d7b6cb6104b0a18dc'
nersc_repository = None

In [2]:
# Load Gaia and connect to girder using NEWT session id
import geolib
girder = geolib.connect(girder_url='http://turtleland4:8080', newt_sessionid=newt_sessionid)

# Select dataset from ESS-DIVE

In [3]:
# Todo select item from ESS-DIVE

# As a surrogate, use file that was uploaded manually
resource_type = 'item'
resource_id = '5c88076206a1b5013e1c6d77'    # /user/admin/Public/local/SFBay_grayscale.tif
girder_url = 'girder://{}/{}'.format(resource_type, resource_id)

essdive_object = geolib.create(girder_url)

In [4]:
# Get bounds
# metadata = essdive_object.get_metadata()
metadata = {
 'bounds': {'coordinates': [[[-122.71879201711467, 37.45219874192699],
    [-122.71879201711467, 38.052231141926995],
    [-122.11875961711466, 38.052231141926995],
    [-122.11875961711466, 37.45219874192699],
    [-122.71879201711467, 37.45219874192699]]],
  'type': 'Polygon'},
 'height': 4323,
 'width': 4323}

# Create polygon to display bounds
bounds = metadata.get('bounds')
coordinates = bounds.get('coordinates')
import geojson
bounds_geometry = geojson.Polygon(coordinates)
bounds_feature = geojson.Feature(geometry=bounds_geometry, properties={'fillColor': 'black', 'fillOpacity': 0.2})
bounds_object = geolib.create(bounds_feature)

# Generate small crop geometry
coords = coordinates[0]
xvals, yvals = zip(*coords)
xmin = min(xvals); ymin = min(yvals)
xmax = max(xvals); ymax = max(yvals)
x = (xmin + xmax) / 2.0
y = (ymin + ymax) / 2.0

# Use small percentage of height & width
dx = 0.12 * (xmax - xmin)
dy = 0.16 * (ymax - ymin)
poly = [[
    [x,y], [x+dx,y+dy], [x-dx,y+dy], [x-dx,y-dy], [x+dx,y-dy]
]]
crop_geometry = geojson.Polygon(poly)
crop_feature = geojson.Feature(geometry=crop_geometry, properties={'fillColor': 'magenta', 'fillOpacity': 0.5})
crop_object = geolib.create(crop_feature)

scene1 = geolib.show([crop_object, bounds_object])
display(scene1)


scene(center={'x': 0.0, 'y': 0.0}, layout=Layout(align_self='stretch', height='400px'))

In [5]:
import ipywidgets as widgets
z = widgets.FloatSlider(min=1, max=16, value=scene1.zoom, layout=dict(width='95%'))
newlink = widgets.jslink((z, 'value'), (scene1, 'zoom'))
display(z)

FloatSlider(value=9.328175, layout=Layout(width='95%'), max=16.0, min=1.0)

# Submit preprocesssing job on NERSC (Cori)

In [6]:
# To run on Girder:
# cropped_object = gaia.crop(essdive_object, crop_object)

# To run on NERSC
import getpass
while not nersc_repository:
    nersc_repository = getpass.getpass('Enter NERSC repository (account): ')
cori_job = geolib.submit_crop(essdive_object, crop_object, nersc_repository)
cori_job

Enter NERSC repository (account):  ·····


user {'groups': [], 'status': 'enabled', 'size': 1125534158, 'emailVerified': True, 'login': 'johnt', 'lastName': 'Tourtellott', '_modelType': 'user', 'public': True, 'admin': True, 'groupInvites': [], 'email': 'john.tourtellott@kitware.com', '_id': '5a60e9de0640fd01195132e4', '_accessLevel': 2, 'created': '2018-01-18T18:39:26.139000+00:00', 'firstName': 'John'}
Creating cluster on cori
Creating SLURM script geolib
script_id 5c8bec8b0640fd012102b8cd
Creating job geolib
Created job folder fac10aaefabc4903b07ce907dd79879c
Created job_id 5c8bec8b0640fd012102b8d1
Uploading geometry file
Submitting job
submit_job body: {'maxWallTime': {'minutes': 5, 'seconds': 0, 'hours': 0}, 'constraint': 'knl', 'numberOfNodes': 1, 'machine': 'cori', 'account': 'm2690', 'jobOutputDir': '/global/cscratch1/sd/johnt/geolib/190315/geolib', 'queue': 'debug'}
Submitted job 5c8bec8b0640fd012102b8d1


'5c8bec8b0640fd012102b8d1'

# Use job id to create GeoLib object

In [3]:
job_id = input('Enter girder job id: ')

Enter girder job id:  5c8bec8b0640fd012102b8d1


In [6]:
geolib_url = girder.lookup_url(job_id=job_id)
output_object = geolib.create(geolib_url)
print('output_object', output_object)


output_object <gaia.girder_data.GirderDataObject object at 0x7f0f1a3ac8d0>


In [ ]:
# Todo - display cropped dataset - need girder instance with geometa and large-image
# geolib.show(output_object)
